In [ ]:
import torch
import numpy as np
import os
import glob
from brainscore_vision import load_model

In [ ]:
model = load_model('resnet50_tutorial')
act_model = model.activations_model

In [4]:
layers = ['layer2', 'layer2', 'layer2', 'layer3']

In [ ]:
imagenet = f'{ os.getcwd() }/data/imagenet/val'

In [5]:
stimuli = glob.glob(os.path.join(imagenet, '**'), recursive=True)
stimuli = [ stimulus for stimulus in stimuli if os.path.isfile(stimulus) ]

In [ ]:
output = f'{ os.getcwd() }/data/output/resnet-20'

In [ ]:
# Set batch size for processing
batch_size = 1000

# Initialize counter for file naming
counter = 1

# Loop through the stimuli in batches
for batch_start in range(0, len(stimuli), batch_size):
  batch = stimuli[batch_start:batch_start + batch_size]
  model_extraction = act_model(batch, layers)
  layers = model_extraction['layer'].values
  features_by_layer = {
    layer: model_extraction.sel(neuroid=model_extraction['layer'] == layer).data
    for layer in layers
  }
  npz_data = {
    layer_name: features 
    for layer_name, features in features_by_layer.items()
  }
  np.savez(os.path.join(output, '{}-{}.npz'.format(model.identifier, counter)), **npz_data)
  counter += 1